In [1]:
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn

In [2]:
resnet50 = models.resnet50(pretrained=True)
resnet50.eval()


/home/aaryan/miniconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/aaryan/miniconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /home/aaryan/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 97.8M/97.8M [00:04<00:00, 23.7MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [3]:
for i, x in resnet50.named_children():
    print(f'NAME: \n {i}')
    print(f'CONTENT: \n {x}')


NAME: 
 conv1
CONTENT: 
 Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
NAME: 
 bn1
CONTENT: 
 BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
NAME: 
 relu
CONTENT: 
 ReLU(inplace=True)
NAME: 
 maxpool
CONTENT: 
 MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
NAME: 
 layer1
CONTENT: 
 Sequential(
  (0): Bottleneck(
    (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (downsample): Sequential(
      (0): Conv2

In [4]:
embeds = {}
def get_activation(name):
    def hook(model, input, output):
        embeds[name] = output
    return hook


In [5]:
for name, layer in resnet50.named_children():
    if name in ['layer1', 'layer2', 'layer3', 'layer4']:
        for i, bottleneck in enumerate(layer.children()):
            hook1 = bottleneck.conv1.register_forward_hook(get_activation(f'conv1_{name}_{i}'))
            hook2 = bottleneck.conv2.register_forward_hook(get_activation(f'conv2_{name}_{i}'))
            hook3 = bottleneck.conv3.register_forward_hook(get_activation(f'conv3_{name}_{i}'))

for hook in [hook1, hook2, hook3]:
    hook.remove()


In [7]:
og = torch.load('og.pt')

2024-04-13 12:59:23.951214: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-13 12:59:26.482286: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-04-13 12:59:30.310541: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-13 12:59:30.313615: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. 

In [9]:
outputs = resnet50(og[0])

TypeError: conv2d() received an invalid combination of arguments - got (tensorflow.python.framework.ops.EagerTensor, Parameter, NoneType, tuple, tuple, tuple, int), but expected one of:
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, tuple of ints padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!tensorflow.python.framework.ops.EagerTensor!, !Parameter!, !NoneType!, !tuple of (int, int)!, !tuple of (int, int)!, !tuple of (int, int)!, int)
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, str padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!tensorflow.python.framework.ops.EagerTensor!, !Parameter!, !NoneType!, !tuple of (int, int)!, !tuple of (int, int)!, !tuple of (int, int)!, int)


In [18]:
model_ft = models.resnet18(pretrained=True)
### strip the last layer
feature_extractor = torch.nn.Sequential(*list(model_ft.children())[:-1])
### check this works
x = torch.randn([1,3,224,224])
print(x.shape)

# print(x)
print(og[0].shape)
# print(og)
output = feature_extractor(x) # output now has the features corresponding to input x
print(output.shape)


torch.Size([1, 3, 224, 224])
(3, 128, 128)
torch.Size([1, 512, 1, 1])


In [23]:
output = feature_extractor(torch.tensor(og[0].numpy()).unsqueeze(0))

In [30]:
output[0,:,0,0]

tensor([0.8955, 0.8725, 0.9530, 1.1116, 0.9396, 0.8213, 1.0376, 1.0744, 0.9455,
        1.0300, 0.9666, 0.8756, 0.9703, 0.9519, 0.9949, 0.9468, 0.8550, 1.3886,
        0.9406, 0.8954, 0.9839, 1.1632, 1.0004, 1.0627, 0.9603, 0.9697, 1.0348,
        0.9479, 0.8058, 0.8880, 0.9234, 0.8880, 0.9104, 0.9108, 0.9157, 0.8803,
        1.0329, 0.9497, 0.9175, 0.8940, 0.8779, 0.8853, 0.8611, 1.0104, 0.8274,
        0.9151, 0.9070, 1.2684, 1.0327, 0.8761, 0.9055, 1.0372, 0.8914, 0.8717,
        1.0325, 0.9843, 1.0860, 0.9242, 1.0257, 0.9311, 1.0219, 0.9835, 0.8617,
        0.8973, 0.9147, 1.0608, 0.8075, 0.8393, 0.9967, 0.9026, 1.0153, 0.8096,
        1.0182, 0.8886, 1.0037, 0.8912, 0.7255, 0.8494, 0.7868, 0.9082, 0.8994,
        1.0852, 1.0533, 1.2141, 0.8672, 0.9341, 0.9112, 0.9667, 0.9975, 0.8606,
        0.7788, 0.9742, 0.9600, 0.8433, 0.9632, 0.9164, 1.0505, 0.9098, 0.8857,
        0.9529, 0.9661, 1.0345, 1.0381, 1.0457, 1.0095, 0.7996, 0.9816, 0.9560,
        0.8933, 0.9620, 0.8735, 0.9576, 